In [ ]:
import json 
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore
import os
from dotenv import load_dotenv
import time
load_dotenv()

In [3]:
from utils1 import openJson,saveJson

## Filtering 1

- videosF1.json contain the videos with fr-FR on the language parameter

In [ ]:
videosR3 = openJson("../jsons/videosR3.json")
len(videosR3)

In [ ]:
videosF1 = []
for video in videosR3:
    if video['langue']== 'fr-FR':
        videosF1.append(video)
saveJson("../jsons/videosF1.json",videosF1)
len(videosF1)

In [ ]:
videosF1 = openJson("../jsons/videosF1.json")
len(videosF1)

## Filtering 2

### Detecter les videos des Chaines TV 

In [ ]:
chainesTv = [
    "France 2", "France 3", "France 4","France 5","Franceinfo",
    "BFMTV", "C8", "CStar", "Gulli", "Cnews",
    "Canal+", "Planète+", "LCI", "Paris première",
    "6ter", "Arte", "M6", "W9",
    "TFX", "TMC", "NRJ12", "TF1","La Chaîne parlementaire",
    "Chérie 25", "RMC"
]
chainesTv = [nomTV.lower().replace(" ", "") for nomTV in chainesTv]

print(len(chainesTv))
print(chainesTv)

In [ ]:
videosF1 = openJson("../jsons/videosF1.json")
len(videosF1)

In [ ]:
channels = openJson("../../collecting/jsons/channels.json")
def getChannelName(channelId):
    for channel in channels:
        if channel['id_chaine']==channelId:
            return channel['nom_chaine'].lower().replace(" ", "")

In [ ]:
chainesTVCounter = Counter()
videosChainesTV = []
for video in videosF1:
    channelName = getChannelName(video['id_chaine'])
    for name in chainesTv:
        if name in channelName:  
            print(name,' ',channelName,' ',video['id_video'])
            chainesTVCounter[name]+=1
            videosChainesTV.append(video['id_video'])

In [ ]:
chainesTVCounter

In [ ]:
import matplotlib.pyplot as plt

labels = list(chainesTVCounter.keys())
sizes = list(chainesTVCounter.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, sizes, color='skyblue')

plt.title("Répartition des vidéos des chaînes TV", fontsize=14)
plt.xlabel("Chaînes TV", fontsize=12)
plt.ylabel("Nombre de vidéos", fontsize=12)
plt.xticks(rotation=45, ha='right')

total_videos_tv = sum(sizes)
plt.text(len(labels) - 2, max(sizes) * 0.8, f"Total : {total_videos_tv} vidéos", fontsize=12, ha='center')
plt.text(len(labels) - 2, max(sizes) * 0.74, "4.62% d'ensemble des vidéos", fontsize=12, ha='center')

# Ajouter les nombres au-dessus des barres et les rendre en gras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f'{int(yval)}', ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
(218/len(videosF1))*100

### Creer videosF2.json

In [ ]:
videosChainesTV

In [ ]:
len(videosChainesTV)

In [ ]:
videosF2 = []
for video in videosF1:
    if video['id_video'] not in videosChainesTV:
        videosF2.append(video)
saveJson("../jsons/videosF2.json",videosF2)

verification

In [ ]:
len(videosF1)-len(videosF2)

In [ ]:
len(videosF1)

## Filtering 3

### With Embedding model

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dangvantuan/sentence-camembert-base")

In [21]:
from sentence_transformers import util

def calculateSimilarity(videoMD, documents):
    embedding_video = model.encode([videoMD], convert_to_tensor=True, normalize_embeddings=True)
    embeddings_documents = model.encode(documents, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(embedding_video, embeddings_documents)[0]
    simScore = scores.mean().item()
    return simScore


In [ ]:
Vocabulary = openJson("../jsons/Voc.json")
len(Vocabulary)

In [23]:
def construire_phrases(mots_cles, taille=25):
    phrases = []
    for i in range(0, len(mots_cles), taille):
        groupe = mots_cles[i:i+taille]
        phrase = ' '.join(groupe)
        phrases.append(phrase)
    return phrases

documents = construire_phrases(Vocabulary)

In [ ]:
print(len(documents))
print(documents[8])

In [ ]:
requete = "Mes livres préférés pour l'AUTONOMIE/AUTOSUFFISANCE : potager, élevage, cuisine, transfo ect)"

calculateSimilarity(requete,documents)

In [26]:
videosF2 = openJson("../jsons/videosF2.json")
scores = []
for video in tqdm(videosF2):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,documents)
    print(score,'  ',video['id_video'])
    scores.append(
        {
            'id_video':video['id_video'],
            'similarity':score
        }
        )


 24%|██▍       | 1091/4504 [18:13<54:50,  1.04it/s]

0.4031078815460205    ybRHmx3ZnEs


 24%|██▍       | 1092/4504 [18:14<54:45,  1.04it/s]

0.29167455434799194    sXOUONI5as8


 24%|██▍       | 1093/4504 [18:15<53:50,  1.06it/s]

...

0.35788577795028687    Kceo8Q6WbBg


 24%|██▍       | 1102/4504 [18:23<51:29,  1.10it/s]

0.39416396617889404    __5cfHpaXfU


 24%|██▍       | 1102/4504 [18:24<56:50,  1.00s/it]


KeyboardInterrupt: 

In [28]:
saveJson("../jsons/keywordsVsMTD.json",scores)


 json saved


In [30]:
videosNon = openJson("../jsons/llmVideosNon.json")
keywordsVsMTD = openJson("../jsons/keywordsVsMTD.json")

In [36]:
Sum = []
for video in videosNon:
    for result in keywordsVsMTD:
        if result['id_video']==video['id_video']:
            print(result['similarity'])
            Sum.append(result['similarity'])
    

0.3045061230659485
0.08022268116474152
0.2510215640068054
0.13914254307746887
0.3456891179084778
0.35787028074264526
0.1502985656261444
0.16819684207439423
0.1344427466392517
0.2256837785243988
0.23916630446910858
0.2077063024044037
0.20742538571357727
0.14259026944637299
0.23402783274650574
0.15943597257137299
0.12071476131677628
0.1028195172548294
0.06876079738140106
0.18382926285266876
0.1969488561153412
0.2574109435081482
0.4151611924171448
0.10599108785390854
0.13396696746349335
0.11021538823843002
0.17402228713035583
0.24198539555072784
0.31872421503067017
0.22191984951496124
0.1624375879764557
0.025443408638238907
0.18030989170074463
0.16425389051437378
0.19237956404685974
0.22844040393829346
0.20870497822761536
0.2546618580818176
0.07434268295764923
0.023086562752723694
0.359218031167984
0.1608438789844513
0.34426793456077576
0.16758760809898376
0.11607363075017929


In [40]:
nn = 0
for fuck in Sum:
    nn+=fuck
print(nn/len(Sum))

0.19248774987128045


In [18]:
contexte_autosuffisance = [
    "Potager familial productif pour l'autonomie en légumes",
    "Élevage de poules, moutons et canards pour l’autonomie alimentaire",
    "Utilisation de plantes médicinales et aromatiques cultivées sur place",
    "Récupération et filtration de l'eau via un puits et osmose inverse",
    "Construction de bacs permacoles faits maison pour une culture efficace",
    "Transformation du sol en terre fertile pour maximiser les rendements",
    "Vie en sobriété heureuse et respect de la nature",
    "Utilisation de semences locales et choix variétal réfléchi",
    "Production locale de viande, œufs et légumes",
    "Utilisation d'une serre pour prolonger les cultures",
    "Métiers artisanaux (ébénisterie, photographie culinaire) intégrés dans le mode de vie",
    "Conception d’un lieu de vie en autonomie énergétique et alimentaire"
]


In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
scores = {}
for video in tqdm(videosF2):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,documents)
    print(score,'  ',video['id_video'])
    scores[video['id_video']]=score


In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
scores = {}
for video in tqdm(videosF2):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,contexte_autosuffisance)
    print(score,'  ',video['id_video'])
    scores[video['id_video']]=score


### LLM Scoring

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
gemini_flash_1 = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY_1"))
gemini_flash_2 = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY_2"))

In [ ]:
sysprompt = """
# La definition d'autossuffinnce

L'autosuffisance est la démarche visant à acquérir la capacité de subvenir par soi-même à ses besoins fondamentaux, 
en premier lieu alimentaires par l'autoconsommation – c'est-à-dire produire, récolter et conserver un maximum de sa propre nourriture, 
souvent en privilégiant le bio, le local et le saisonnier. Plus qu'une simple recherche d'autonomie matérielle, 
elle représente un engagement pour être moins dépendant du système économique et social extérieur, impliquant des choix concrets comme trouver un lieu propice et le concevoir judicieusement (par exemple en permaculture), 
ainsi qu'un changement dans la manière de valoriser son temps et de consommer, formant ainsi le fondement d'une vie plus autonome.

---

# Votre mission

- Décider si une vidéo concerne la thématique de l'autosuffisance en se basant sur ses métadonnées (titre, description, tags).  
- Les vlogs, les interviews et les vidéos de partage d'expériences en lien avec l'autosuffisance sont intéressants.  
- Les tutoriels et les vidéos de conseils sont également pertinents.  
- Les vidéos présentant une technique liée à l'autosuffisance ou y contribuant sont utiles.  
- Attribuer un score entier de 1 à 10 pour évaluer la pertinence de la vidéo par rapport à la thématique de l'autosuffisance.

---

# Votre réponse  

- La réponse doit être au format JSON :  
  
    "decision": "oui ou non",
    "justification": "Justifiez votre décision avec des arguments"
    "score": votre èvaluation
 
"""

userprompt = """
Titre
---
{titre}
Description
---
{description}
tags
---
{tags}
"""

prompt = ChatPromptTemplate([
    ("system", sysprompt),
    ("user", userprompt)
])

chain_1 = prompt | gemini_flash_1
chain_2 = prompt | gemini_flash_2


- Test

In [ ]:
input = {"titre":"Tuteurer et redresser un arbre 🌳",
         "description":"Dans cette vidéo nous allons voir ensemble comment mettre un tuteur afin de redresser un arbre qui penche suite à la tempête.\nCette vidéo vous aideras pour placer un tuteur en travers , sangler un arbre selon le sens du vent,\n\nIl existe plusieurs facon de tuteurer un arbre, haubanage, tripode,.. mais dans le cas présent un tuteur en biais suffiras puisque l'arbre n'est pas trés gros.\n\n\n▬▬▬▬▬▬▬▬▬▬▬ POUR ALLER PLUS LOIN  ▬▬▬▬▬▬▬▬▬▬▬\n\n💡 Abonnez-vous pour me soutenir 🧡\n\n\n💡Rejoignez-nous sur twitter\net tik tok\n\n\n▬▬▬▬▬▬▬▬▬▬▬ CHAPITRES DE CETTE VIDÉO ▬▬▬▬▬▬▬▬▬▬\n\n00:00 : Introduction\n00:23 : Mise en place tuteur\n01:54 : Sanglage et Redresser l'arbre\n02:54 : Sens du vent et détails tuteurage\n\n▬▬▬▬▬▬▬▬▬▬▬ A PROPOS DE CETTE VIDÉO ▬▬▬▬▬▬▬▬▬▬▬\n\nLe Jardin du moulin\nFrancois\n#potager #jardindumoulin",
         "tags":', '.join( [
      "TCS",
      "Agriculture Biologique",
      "Bio",
      "GAB 85",
      "FNAB",
      "Vendée",
      "Techniques Culturales Simplifiées",
      "GAEC Les Jonquilles",
      "ECOPHYTO",
      "Demain Vendée",
      "Conservation des sols",
      "Sol vivant",
      "Productions végétales",
      "Maïs"
    ])}
print(input)
print(chain_2.invoke(input))

In [ ]:
def cleanAnswer(answer):
    answer = answer.strip("`")   # Supprimer tous les backticks
    if answer.startswith("json"):
        answer = answer[4:].strip() 
    return  answer

In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
llmEvaluation = []
count1 = 0
count2 = 0
temp = 0

for video in tqdm(videosF2):
    input = {"titre":video['titre_video'],
             "description":video['description'],
             "tags":', '.join(video['tags']),}
    try:
        if count1  <= 13:
            print("KEY 1")
        
            answer = cleanAnswer(chain_1.invoke(input))
            #print(video['id_video'],'/n',answer)
            evaluation = {}
            evaluation['id_video']=video['id_video']
            evaluation.update(json.loads(answer))
            llmEvaluation.append(evaluation)
            
            count1+=1
            temp+=1            
             
            print("count1 ",count1)  
            
        if count1  > 13 and count2 <= 13:
            print("KEY 2")
            
            answer =  cleanAnswer(chain_2.invoke(input))
            #print(video['id_video'],'/n',answer)
            evaluation = {}
            evaluation['id_video']=video['id_video']
            evaluation.update(json.loads(answer))
            llmEvaluation.append(evaluation)
            
            count2+=1
            temp+=1
            print("count2 ",count2)  
            
        if count1 > 13 and count2 > 13 :
            print("sleep for 1 min")
            time.sleep(60)
            count1 = 0
            count2 = 0
        

        if temp >= 100:
            saveJson("../jsons/llmEvaluation.json",llmEvaluation)
            temp=0
            
    except Exception as e:
        print(f"error {e}")

- Seperate the videos classified as not relevent

In [6]:
llmEvaluations = openJson("../jsons/llmEvaluation.json")
videosF2 = openJson("../jsons/videosF2.json")

def getNonVideo(ID):
    for video in videosF2:
        if video['id_video']==ID:
            return video

In [ ]:
len(videosF2)

In [ ]:
len(llmEvaluations)

In [ ]:
videosNon = []
for evaluation in llmEvaluations:
    try:

        if evaluation['decision'] =='non':
            videoNo = getNonVideo(evaluation['id_video'])
            videoNo.update(evaluation)
            videosNon.append(videoNo)
    except:
        print(evaluation)
        break
    
saveJson("../jsons/llmVideosNon.json",videosNon)

In [ ]:
videosNon = openJson("../jsons/llmVideosNon.json")
len(videosNon)